# Generating Simulated Deformation Events

Simulated Deformation Events are modeled from a finite rectanglar source using the method from: Okada, Surface deformation due to shear and tensile faults in a half-space.

Okada's Paper:  https://www.bosai.go.jp/e/pdf/Okada_1985_BSSA.pdf <br>
Original Basis: https://github.com/matthew-gaddes/SyInterferoPy

### Imports

In [1]:
# @title
!pip install geopy git+https://github.com/pvigier/perlin-numpy
!pip install astropy
%cd ..
%pwd

  Cloning https://github.com/pvigier/perlin-numpy to /tmp/pip-req-build-zui8gk09
  Running command git clone --filter=blob:none --quiet https://github.com/pvigier/perlin-numpy /tmp/pip-req-build-zui8gk09
  Resolved https://github.com/pvigier/perlin-numpy to commit 5e26837db14042e51166eb6cad4c0df2c1907016
  Preparing metadata (setup.py) ... done
/home/jovyan/AI-Event-Monitoring


'/home/jovyan/AI-Event-Monitoring'

In [2]:
# @title
import numpy as np
import matplotlib.pyplot as plt
import astropy

from insar_eventnet.sarsim import (
    Okada,
    gen_gaussian_noise,
    coherence_mask_simulate,
    gen_simulated_deformation,
    gen_sim_noise,
    aps_simulate,
    gen_fake_topo,
    atm_topo_simulate,
)

plt.rcParams["figure.figsize"] = (16, 8)


def display_two_images(unwrapped, wrapped):
    fig, [axs_unwrapped, axs_wrapped] = plt.subplots(1, 2)

    axs_unwrapped.set_title("Unwrapped")
    axs_wrapped.set_title("Wrapped")

    axs_unwrapped.imshow(unwrapped, cmap="jet")
    axs_wrapped.imshow(wrapped, cmap="jet")


def display_three_images(unwrapped, wrapped, masked):
    fig, [axs_unwrapped, axs_wrapped, axs_masked] = plt.subplots(1, 3)

    axs_unwrapped.set_title("Unwrapped")
    axs_wrapped.set_title("Wrapped")
    axs_masked.set_title("Mask")

    axs_unwrapped.imshow(unwrapped, cmap="jet")
    axs_wrapped.imshow(wrapped, cmap="jet")
    axs_masked.imshow(masked, cmap="jet")

ModuleNotFoundError: No module named 'astropy'

## Generate Event Manually

The Okada Class allows you to manually generate simulated event's using a dict of parameters. The simulation is done using Okada's Surface Deformation Due to Shear and Tensile Faults in a Half-Space Model, assuming a finite rectangular source. The kwargs dict below highlights all of the necessary parameters.

Ultimately, it's purpose for the EventNet project is to generate the line-of-sight displacement (self.los_displacement). But it also generates quite of bit of information that you may access:
```python
self.source_type
self.source_x
self.source_y
self.tile_size
self.params
self.x_axis_shape
self.y_axis_shape
self.grid_x
self.grid_y
self.los_vector
self.lames_mu       # μ
self.lames_lambd    # λ
self.nu             # ν (poisson ration)
self.length
self.strike
self.dip            # δ
self.opening
self.slip
self.rake
self.width
self.depth
self.east
self.north
self.okada_x        # ξ
self.okada_y
self.d
self.q
self.eta            # η
self.U1
self.U2
self.U3
self.displacement
self.los_displacement
```

### Setup Input Parameters

In [ ]:
seed = (
    0  # 0 seed here means random atmospheric noise every time the function is called.
)
tile_size = 512
event_type = "quake"

source_x = 20000  # min_x, max_x is 0->45000 at (512, 512)
source_y = 20000  # min_y, max_y is 0->45000 at (512, 512)

kwargs = {
    "strike": 180,  # for source_type 'quake'
    "dip": 45,  # for source_type 'quake'
    "length": 4000,  # for source_type 'quake'
    "rake": 90,  # for source_type 'quake'
    "slip": 2,  # for source_type 'quake'
    "top_depth": 4000,  # for source_type 'quake'
    "bottom_depth": 8000,  # for source_type 'quake'
    "width": 2000,  # for source_type 'sill' and 'dyke'
    "depth": 4000,  # for source_type 'sill' and 'dyke'
    "opening": 0.5,  # for source_type 'sill' and 'dyke',
    "source_x": source_x,
    "source_y": source_y,
}

#### Get Displacement and Amplify

Since the model gives us line-of-sight displacement, the values are generally very low and won't even get close to wrapping around pi. Because of this, we need to scale the values for them to be useful for our purposes.

In [ ]:
Event = Okada(event_type, (source_x, source_y), tile_size=tile_size, **kwargs)

scalar = 100 * np.pi
los_displacement = Event.los_displacement
phase = scalar * los_displacement
wrapped_phase = np.angle(np.exp(1j * (phase)))

display_two_images(phase, wrapped_phase)

## Manually Add Simulated Noise and Error Sources

Currently, there three primary types of error that are supported: turbulant atmospheric error, topographic atmospheric error, and guassian noise. Additionally, it is possible to generate masked areas of incoherence. 

### Static-Like Guassian Noise

In [ ]:
seed = 0
tile_size = 512
noise_level = 0.5
threshold = 0.5
noise_grid = gen_gaussian_noise(
    seed, tile_size, noise_level=noise_level, threshold=threshold
)

event_with_gaussian = noise_grid + phase

gaussian_wrapped_grid = np.angle(np.exp(1j * (event_with_gaussian)))

display_two_images(event_with_gaussian, gaussian_wrapped_grid)

### Inconsistant Gaussian Noise

Due to limitations in the ability to mask out incoherent areas in interferograms, there can be splotchy blips of of static-like noise. In order to emulate this, two gaussian noise grids are generated. The threshold value is then used with one of the noise grids to determine what noise from the other can make it through. This means that the threshold value should be some fraction of the noise_level value. When the threshold equals the noise_level all noise is let through.

In [ ]:
seed = 0
tile_size = 512
noise_level = np.pi
threshold = np.pi / 8
noise_grid = gen_gaussian_noise(
    seed, tile_size, noise_level=noise_level, threshold=threshold
)

inc_gaussian_wrapped_grid = np.angle(np.exp(1j * (noise_grid)))

display_two_images(noise_grid, inc_gaussian_wrapped_grid)

### Simulated Masked-out Areas of Incoherence

In [ ]:
threshold = 0.2
coherence_mask = coherence_mask_simulate(tile_size, threshold=threshold)
coh_indices = coherence_mask[0, 0:tile_size, 0:tile_size] == 0

phase_with_coh_mask = np.copy(phase)
phase_with_coh_mask[coh_indices] = 0
wrapped_with_coh_mask = np.angle(np.exp(1j * phase_with_coh_mask))

display_two_images(phase_with_coh_mask, wrapped_with_coh_mask)

### Turbulant Atmospheric Error

In [ ]:
atmosphere_scalar = 100 * np.pi

turb_phase = aps_simulate(tile_size) * atmosphere_scalar
turb_event_phase = turb_phase + phase
wrapped_turb_phase = np.angle(np.exp(1j * turb_event_phase))

display_two_images(turb_event_phase, wrapped_turb_phase)

### Topographic Atmospheric Error

In [ ]:
atmospheric_scalar = 100 * np.pi

simulated_topography = gen_fake_topo(size=tile_size)

topo_phase = np.abs(atm_topo_simulate(simulated_topography) * atmosphere_scalar * np.pi)
topo_event_phase = topo_phase + phase
wrapped_topo_phase = np.angle(np.exp(1j * (topo_event_phase)))

display_two_images(topo_event_phase, wrapped_topo_phase)

### Combining All Error Sources

In [ ]:
combined_error_phase = phase + topo_phase + turb_phase + noise_grid
combined_error_wrapped_phase = np.angle(np.exp(1j * combined_error_phase))

display_two_images(combined_error_phase, combined_error_wrapped_phase)

## Generating Training-Optimized Events

Deep Learning models can be extremely robust, and they can learn from deep nuances in data. Like humans, deep learning algorithms learn much more effectively when information is presented in certain ways. This is especially true here! For example, if we allow any possible valid combination to be generated, we may be generating events which don't produce enough deformation to show through the background noise. At best, this would waste space and time, since the model has no information to learn from. At worst, the model could incorrectly learn to identify noise as an event due to the positive label that would be attached. Therefore, gen_simulated_deformation randomly selects parameters from ranges which properly highlight the most important features for identification purposes. Furthermore, a boundary between portions of events is added, and the wrapped images are normalized between 0 and 1.

Events from gen_simulated_deformation are used as the ***positives*** for training purposes.

Events can also be produced with user-provided kwargs if desired.

In [ ]:
seed = 123456
tile_size = 512
event_type = "quake"
log = True

#### With Kwargs

Including our kwargs will create the event using the specified options. Please note that all of the dict keys need to be present when using kwargs. It is also important to note that there is randomly generated atmospheric noise, using an fft method; thus, if you want the same image every time, you need to use a seed.

In [ ]:
unwrapped, masked, wrapped, event_is_present = gen_simulated_deformation(
    seed=seed, tile_size=tile_size, log=log, event_type=event_type, **kwargs
)

display_three_images(unwrapped, wrapped, masked)

#### With Random Parameters

The ranges for the randomly generated parameters do not cover all possible valid combinations; rather, they are set to reliably produce events that are suitible for eventnet's training.

In [ ]:
unwrapped, masked, wrapped, event_is_present = gen_simulated_deformation(
    seed=seed,
    tile_size=tile_size,
    log=log,
    event_type=event_type,
)

display_three_images(unwrapped, wrapped, masked)

## Randomly Generating Training-Optimized Error

Similar to the images with events, the ***negatives*** also benefit from curation. This is done using the gen_sim_noise function. 

In [ ]:
seed = 123456
tile_size = 512
gaussian_only = False  # If no atmospheric noise is desired.
atmosphere_scalar = 200 * np.pi  # Again, the 'displacement' needs to be scaled.

unwrapped, masked, wrapped, event_is_present = gen_sim_noise(
    seed=seed, tile_size=tile_size, gaussian_only=False, atmosphere_scalar=200 * np.pi
)

print("Max, Min Noise Values: ", np.max(unwrapped), np.min(unwrapped))

display_three_images(unwrapped, wrapped, masked)